<a href="https://colab.research.google.com/github/orionhunts-ai/research/blob/main/Cyber_2_SQL_phi_small_8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# @title HF Git { run: "auto" }
#!git init
!git config --global user.email "core@synavate.tech"
!git config --global user.name "Snyata"
!git remote set-url origin https://wwww.huggingface.co/synavate/cyber-tuned-phi3-8k.git
!git add ./**
!git commit -m "Checkpoint 1 - Datset Cleaned"
!git push -u origin master

error: open("drive/MyDrive/AWS CPP/AWS Services in a Single Line.gdoc"): Operation not supported
error: unable to index file 'drive/MyDrive/AWS CPP/AWS Services in a Single Line.gdoc'
fatal: adding files failed
On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	drive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
error: src refspec master does not match any
error: failed to push some refs to 'https://wwww.huggingface.co/synavate/cyber-tuned-phi3-8k.git'


In [2]:
# @title Install Core Libraries { run: "auto", display-mode: "form" }
RUN=0
if RUN == 0:
  %pip -qqq install transformers torch datasets wandb
  %pip -qqq install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
  %pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

  RUN += 1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

# Phine Tuning Microsoft Research Phi-3-small-8k-instruct For Agentic Cyber 👾

  *  Training (and reporting) on Google Colab
   for access to their high powered CUDA and  
  Leveraging [unsloth](https://github.com/unslothai) for efficient fine tuning, and conservative memory utilisation.
    * Dataset for training is using SQL statements that were generated from Synthetic examples text thanks to Gretel.AI (100k examples to be exact!)
    * Sampling from the full set for those that are most relevant to Cyber Security Analysts.***

    * Aside from traditional and evolving Evaluations I will also deploy a number of the finely tuned models in a Microsoft Autogen agentic environment to see how they perform on basic analysis on a database.

    * Red Panda (a high performance streaming data alternative to Kafka will be used)



##  Data and Tool Preparation
**Summary:**

This study explores the fine-tuning of the Phi-3-small-instruct model (7.39 billion parameters) by using Daniel So's Unsloth for a Cyber Threat Intelligence (CTI) task using methods like Parameter-Efficient Fine-Tuning (PEFT), Low-Rank Adaptation (LoRA), and Quantized Low-Rank Adaptation (QLoRA). It aims to evaluate performance degradation, model collaboration in agentic environments, and the potential influence of GPT-4. Synthetic data from gretel.ai was also utilized to supplement the fine-tuning process and enhance data diversity and robustness.

In [3]:
!python -m pip install -qqq huggingface_hub
wandb.login(key=userdata.get('WANDB_API_KEY'))

NameError: name 'wandb' is not defined

In [ ]:
#ML Ops and EDA Imports
import os
import wandb
import torch
import pandas as pd
import numpy as np
from google.colab import userdata

wandb.init(project="Cyber-Phi-Small-8k", )
!huggingface-cli login --token $HF_TOKEN
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# Loading GretelAI's Synthetic Dataset - Create your own at https://gretel.ai/'''

from datasets import load_dataset
ds = load_dataset("gretelai/synthetic_text_to_sql")
df = ds['train'].to_pandas()
df.head()

In [ ]:
# prompt: Using dataframe df: make a dataframe called df_domain with only rows in the df_domains list and perform other data cleaning on the df_domain dataframe

# Assuming df_domains is a list of domains to filter by
df_domains = ['artificial intelligence','defense industry', 'defense operations', 'defense security', 'cybersecurity',
              'defence contractors', 'technology']
df_domain = df[df['domain'].isin(df_domains)].copy()


df_domain = df_domain.drop_duplicates()
df_domain = df_domain.dropna()
df_domain.head()


In [ ]:
# prompt: a vertical bar chart showing the distribution of domains in the dataframe

import matplotlib.pyplot as plt

# Assuming 'domain' is a column in your dataframe
domain_counts = df_domain['domain'].value_counts()

plt.figure(figsize=(10, 6))
domain_counts.plot(kind='bar')
plt.xlabel('Domains')
plt.ylabel('Count')
plt.title('Distribution of Domains')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
print(df_domain.shape)
print(df_domain.columns)
columns_to_drop=['sql_complexity', 'sql_complexity_description', 'sql_explanation', 'sql_context']
df_clean = df_domain.drop(columns=columns_to_drop, inplace=False)

*** Data set is relatively balanced therefore there is no need for any manipulation or rebalancing of the domains. Fortunately, cybersecurity is the most well represented - Dataset Ready to go ***

In [ ]:
run = wandb.init(project="Cyber-Phi-Small-8k", job_type="data-prep")
artifact = wandb.Artifact(name="data_set_cleaned", type="data")
artifact.add_file()
run.log_artifact(artifact)
run.finish()



---



In [ ]:
# Load model directly from HuggingFace

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-small-8k-instruct", trust_remote_code=True)

In [ ]:
def load_and_log():

    # 🚀 start a run, with a type to label it and a project it can call home
    with wandb.init(project="artifacts-data-models", job_type="load-data") as run:

        datasets = load()  # separate code for loading the datasets
        names = ["training", "validation", "test"]

        # 🏺 create our Artifact
        raw_data = wandb.Artifact(
            "cyber-phi", type="dataset",
            description="Cyber-Phi",
            metadata={"source": "torchvision.datasets.MNIST",
                      "sizes": [len(dataset) for dataset in datasets]})

        for name, data in zip(names, datasets):
            # 🐣 Store a new file in the artifact, and write something into its contents.
            with raw_data.new_file(name + ".pt", mode="wb") as file:
                x, y = data.tensors
                torch.save((x, y), file)

        # ✍️ Save the artifact to W&B.
        run.log_artifact(raw_data)

load_and_log()



---
###APPENDIX A

### 🤗Fine-Tuning Techniques: 🤗

**PEFT** (Parameter-Efficient Fine-Tuning): Fine-tunes pre-trained models by adjusting only a small subset of parameters, reducing computational costs.

**LoRA** (Low-Rank Adaptation): Enhances transformer models by injecting and training low-rank matrices within each layer, minimizing the number of trainable parameters.

**QLoRA** (Quantized Low-Rank Adaptation): Combines low-rank adaptation with weight quantization to achieve efficient fine-tuning with reduced memory and computational requirements.

**Full Fine-Tuning:** Updates all parameters of the pre-trained model, offering high flexibility at the cost of increased computational resources.

**Distillation:** Trains a smaller model to mimic the behavior of a larger pre-trained model, optimizing efficiency while maintaining performance.